## Importing Modules

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

import warnings
warnings.filterwarnings("ignore")
load_dotenv()

True

## Tools

In [3]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
display(multiply.name,multiply.description,multiply.args)

'multiply'

'Given 2 numbers a and b this tool returns their product'

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# tool binding

In [2]:
llm = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.1 ## this is creative parameter
    )

In [9]:
llm.invoke('hi')

AIMessage(content='Hello! 👋  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 10, 'total_tokens': 24, 'completion_time': 0.025454545, 'prompt_time': 0.00116815, 'queue_time': 0.250363839, 'total_time': 0.026622695}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c4bc10c9-7764-4024-a54a-9ea06291c0df-0', usage_metadata={'input_tokens': 10, 'output_tokens': 14, 'total_tokens': 24})

In [ ]:
llm_with_tools = llm.bind_tools([multiply]) ## tools binding

In [11]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='<tool-use></tool-use>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 968, 'total_tokens': 979, 'completion_time': 0.02, 'prompt_time': 0.018666484, 'queue_time': 0.247497857, 'total_time': 0.038666484}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--77876c35-f14a-4189-9ed6-b6ec89480c7d-0', usage_metadata={'input_tokens': 968, 'output_tokens': 11, 'total_tokens': 979})

In [21]:
display(llm_with_tools.invoke('What is 3 times 4?'),
llm_with_tools.invoke('What is 3 times 4?').tool_calls
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'dtd78yf3q', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 972, 'total_tokens': 1057, 'completion_time': 0.154545455, 'prompt_time': 0.023027552, 'queue_time': 0.250776829, 'total_time': 0.177573007}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ef45dbd6-8862-4548-a88e-f6175cec9b5b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': 'dtd78yf3q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 972, 'output_tokens': 85, 'total_tokens': 1057})

[{'name': 'multiply',
  'args': {'a': 3, 'b': 4},
  'id': '8njfg1f2a',
  'type': 'tool_call'}]

In [28]:
result = llm_with_tools.invoke('What is 3 times 4?')
result.tool_calls[0]['args']

{'a': 3, 'b': 4}

In [29]:
multiply.invoke(result.tool_calls[0]['args'])

12

In [ ]:
multiply.invoke(result.tool_calls[0]) ## we get tool message as well

ToolMessage(content='12', name='multiply', tool_call_id='axx825mcq')

In [31]:
query = HumanMessage('can you multiply 3 with 1000')

In [32]:
messages = [query]

In [33]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [34]:
result = llm_with_tools.invoke(messages)

In [35]:
messages.append(result)

In [36]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tpcw3q1j5', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 975, 'total_tokens': 1063, 'completion_time': 0.16, 'prompt_time': 0.020507033, 'queue_time': 0.247642098, 'total_time': 0.180507033}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--25204bc2-8aa8-43c4-b6ef-fd4d38bd20be-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'tpcw3q1j5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 975, 'output_tokens': 88, 'total_tokens': 1063})]

In [37]:
tool_result = multiply.invoke(result.tool_calls[0])

In [38]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='tpcw3q1j5')

In [39]:
messages.append(tool_result)

In [40]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'tpcw3q1j5', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 975, 'total_tokens': 1063, 'completion_time': 0.16, 'prompt_time': 0.020507033, 'queue_time': 0.247642098, 'total_time': 0.180507033}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--25204bc2-8aa8-43c4-b6ef-fd4d38bd20be-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'tpcw3q1j5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 975, 'output_tokens': 88, 'total_tokens': 1063}),
 ToolMessage(content='3000', name='multiply', tool_call_id='tpcw3q1j5')]

In [41]:
llm_with_tools.invoke(messages).content

'3000'

## Conversion Factor

In [27]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float: ## injected tool arg developer will inject the value after running above tool
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [28]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [29]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754524801,
 'time_last_update_utc': 'Thu, 07 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1754611201,
 'time_next_update_utc': 'Fri, 08 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.7539}

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':87.7539})

877.539

In [12]:
# tool binding
# llm = ChatOpenAI()

llm = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.1 ## this is creative parameter
    )

In [13]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [14]:
llm_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D021FB2FF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D021FFD430>, model_name='Gemma2-9b-It', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculates the target currency value from a given base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'integer'}}, 'required': ['base_currency_value'], 'type'

In [15]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [16]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [17]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
ai_message.tool_calls ## LLM is unable to call tools automatically, so we need to do it manually check follwoing code to do it

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'htw7kc0ta',
  'type': 'tool_call'}]

In [34]:
import json


# Setup LLM

llm = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.1 ## this is creative parameter
    )

# Bind tools
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

# Step 1: Call the LLM to decide which tools to use
messages = [HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]
response = llm_with_tools.invoke(messages)
print("LLM Response:", response.tool_calls)


# Step 2: Run tool(s) based on LLM response
for tool_call in response.tool_calls:
    if tool_call["name"] == "get_conversion_factor":
        print("🔧 Calling Tool:", tool_call["args"])
        rate = get_conversion_factor.invoke(tool_call["args"])
        print("✅ Conversion Rate:", rate['conversion_rate'])

        # Now call convert manually using rate
        converted_value = convert.invoke({
            "base_currency_value": 10,
            "conversion_rate": rate['conversion_rate']
        })
        print("💸 Converted Value (10 INR to USD):", converted_value)



LLM Response: [{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'g5s1zea8z', 'type': 'tool_call'}]
🔧 Calling Tool: {'base_currency': 'INR', 'target_currency': 'USD'}
✅ Conversion Rate: 0.0114
💸 Converted Value (10 INR to USD): 0.114


In [17]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': '4fa87d14z',
  'type': 'tool_call'}]

In [18]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': '4fa87d14z',
  'type': 'tool_call'}]

In [73]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    print("tool_message1")
    print(tool_message1)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    print(f'Conversion rate from USD to INR is {conversion_rate}')
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



tool_message1
content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1754524801, "time_last_update_utc": "Thu, 07 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1754611201, "time_next_update_utc": "Fri, 08 Aug 2025 00:00:01 +0000", "base_code": "INR", "target_code": "USD", "conversion_rate": 0.0114}' name='get_conversion_factor' tool_call_id='ytbnfzprp'
Conversion rate from USD to INR is 0.0114


In [74]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'ytbnfzprp', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 1126, 'total_tokens': 1221, 'completion_time': 0.172727273, 'prompt_time': 0.02652538, 'queue_time': 0.251101329, 'total_time': 0.199252653}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--caec9aba-b0dc-46c8-b119-e0c3250ac3ea-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'ytbnfzprp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1126, 'output_tokens': 

In [55]:
llm_with_tools.invoke(messages).content

''

In [59]:
messages = [HumanMessage('Who is father of cricket')]
ai_message = llm_with_tools.invoke(messages)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sp6tgdk75', 'function': {'arguments': '{"base_currency":"USD","target_currency":"GBP"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 205, 'prompt_tokens': 1107, 'total_tokens': 1312, 'completion_time': 0.372727273, 'prompt_time': 0.025842006, 'queue_time': 0.249077704, 'total_time': 0.398569279}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8ec3bc0f-e459-408d-8edc-ac429d3aa4d1-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'GBP'}, 'id': 'sp6tgdk75', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1107, 'output_tokens': 205, 'total_tokens': 1312})

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to help! How can I assist you today?

> Finished chain.
